In [63]:
import numpy as np

class ActivationFunctions:
        
    def ReLu(data):
        temp = np.array(data)
        return np.maximum(0,temp)
        
    def Sign(data):
        temp = np.array(data)
        return np.sign(temp)
            
    def Sigmoid(data):
        temp = np.array(data)
        return 1/(1+np.exp(-temp))
    def Default(data):
        temp = np.array(data)
        return temp
    def SoftMax(data):
        temp = np.exp(np.array(data))
        return temp/np.sum(temp)
    
class LossFunctions:
    
    def RMSE(estimator,goal):
#         assert isinstance(estimator,np.array)
#         assert isinstance(goal,np.array)
#         print(estimator.shape, goal.shape)
#         assert estimator.shape == goal.shape
#         assert estimator.shape[0] == 1 or estimator.shape[1] ==1
        estimator = np.array(estimator)
        goal = np.array(goal)
        
        return np.sum([i*i for i in (estimator-goal)])/len(np.max([estimator.shape[0] ,estimator.shape[1]]))
    def AE(estimator,goal):
        return (estimator-goal)*(estimator-goal)
        

class Layer:
    
    def __init__(self,shape,function=ActivationFunctions.Default) -> None:
        assert len(shape)==2
        assert isinstance(shape[0],int) and isinstance(shape[1],int)
        assert shape[0]>0 and shape[1]>0
        self.function=function
        self.shape = shape

        self._init_matrix()
        self.output=None

    def __init__(self,shape_x:int,shape_y:int,function=ActivationFunctions.Default) -> None:
        assert shape_y>0 and shape_x>0

        self.shape = (shape_x,shape_y)
        self.function=function
        self._init_matrix()
        self.output=None

    def _init_matrix(self):
        self.martix = np.random.randn(self.shape[0],self.shape[1])
        self.bias=np.random.randn(1,self.shape[1])


    def forward(self,data,function=ActivationFunctions.Default):
        
        if function in [None,ActivationFunctions.Default]:
            function = self.function
        if function == None:
            self.output=np.dot(data,self.martix)-self.bias
            return self.output
        else:
            self.output = function(np.dot(data,self.martix)-self.bias)
            return self.output
    

In [64]:
class Model:

    def __init__(self):
        self.layers=[]
        self.structure=[]
        self.learningFactor=0.02 # TODO implementation
        self.dataset={}
        self.loss_v=None
    def addLayer(self,numberOfNeurons:int,function=ActivationFunctions.Default):
        assert numberOfNeurons>0
        # TODO to validate function
        self.layers.append((numberOfNeurons,function))
        
    def compileModel(self):
        assert len(self.layers)>=2
        
        for i in range(len(self.layers)-1):
            self.structure.append(Layer(self.layers[i][0],self.layers[i+1][0],self.layers[i+1][1]))
            
    def forward(self,data):
        assert len(self.structure)>=2
        output = None
        for i in self.structure:
            output = i.forward(output)
        return output
    
    def upload_test_dataset(self,data): #TODO add validation
        self.dataset['test'] = data
        
    def upload_train_dataset(self,data): #TODO add validation
        self.dataset['train'] = data
    
    def loss(self,goal,function=LossFunctions.RMSE):
        return function(self.structure[-1].output,goal)
    
    def train(self):
        for x,y in zip(self.dataset['train']['egzo'],self.dataset['train']['endo']):
            out = x
            for l in self.structure:
                out = l.forward(out)
            self.loss_v=self.loss(y,LossFunctions.AE)
            

In [68]:
model =Model()
model.addLayer(5,ActivationFunctions.Sigmoid)
model.addLayer(5,ActivationFunctions.Sigmoid)
model.addLayer(1,ActivationFunctions.Sigmoid)

model.compileModel()
train_dataset= {
    'egzo':np.random.randn(10,5),
    'endo':np.random.randn(10,1)
}
#print(train_dataset)
model.upload_train_dataset(train_dataset)
# for i in range(len(model.structure)):
#     print(model.structure[i].shape,end=f" {model.structure[i].function}\n ")
#model.forward(np.array([1,2,3,4,5]))
model.train()
model.loss_v

array([[0.73084789]])

In [67]:
ActivationFunctions.Sign()

TypeError: Sign() missing 1 required positional argument: 'data'

In [42]:
ActivationFunctions.SoftMax(np.linspace(-1,1,100))

1.0000000000000002


Taka jest docelowa funckja LOSS czyli taka którą minimalizujemy.

```1```- indidator function https://www.wikiwand.com/en/Indicator_function

```i``` iteruje sie po obserwacjach

```c``` iteruje sie po klasach

$p_{model} [y_i \in C_c ]$ prawdopodobieństwo, że obserwacja i należy do klasy c

$-1/N*\Sigma_{i=1} ^N \Sigma_{c=1} ^C 1_{y_{i} \in C_{c}} log(p_{model}[y_i \in C_{c}]) $